In [1]:
"""
https://leetcode.com/problems/basic-calculator-iv/

Given an expression such as expression = "e + 8 - a + 5" and an evaluation map such as {"e": 1} (given in terms of evalvars = ["e"] and evalints = [1]), 
return a list of tokens representing the simplified expression, such as ["-1*a","14"]

An expression alternates chunks and symbols, with a space separating each chunk and symbol.
A chunk is either an expression in parentheses, a variable, or a non-negative integer.
A variable is a string of lowercase letters (not including digits.) Note that variables can be multiple letters, and note that variables never have a leading coefficient or unary operator like "2x" or "-x".
Expressions are evaluated in the usual order: brackets first, then multiplication, then addition and subtraction.

For example, expression = "1 + 2 * 3" has an answer of ["7"].
The format of the output is as follows:

For each term of free variables with a non-zero coefficient, we write the free variables within a term in sorted order lexicographically.
For example, we would never write a term like "b*a*c", only "a*b*c".
Terms have degrees equal to the number of free variables being multiplied, counting multiplicity. We write the largest degree terms of our answer first, breaking ties by lexicographic order ignoring the leading coefficient of the term.
For example, "a*a*b*c" has degree 4.
The leading coefficient of the term is placed directly to the left with an asterisk separating it from the variables (if they exist.) A leading coefficient of 1 is still printed.
An example of a well-formatted answer is ["-2*a*a*a", "3*a*a*b", "3*b*b", "4*a", "5*c", "-6"].
Terms (including constant terms) with coefficient 0 are not included.
For example, an expression of "0" has an output of [].
Note: You may assume that the given expression is always valid. All intermediate results will be in the range of [-231, 231 - 1].


Constraints:
1 <= expression.length <= 250
expression consists of lowercase English letters, digits, '+', '-', '*', '(', ')', ' '.
expression does not contain any leading or trailing spaces.
All the tokens in expression are separated by a single space.
0 <= evalvars.length <= 100
1 <= evalvars[i].length <= 20
evalvars[i] consists of lowercase English letters.
evalints.length == evalvars.length
-100 <= evalints[i] <= 100
"""

from collections import defaultdict
def calc(s, evalvars, evalints):
    def _isNum(t):
        for c in t:
            if not c.isdigit():
                return False
        return True

    def _multNums(nums):
        retVal = 1
        for n in nums:
            retVal *= n
        return retVal

    def _getDegrees(s):
        numMult = 0
        for c in s:
            if c == "*":
                numMult
        return numMult + 1

    def _addTerms(terms):
        termMap = {} # defaultdict(int)
        termMap["NUM"] =  [1, 0]

        for t in terms:
            if len(t["VAR"]) + len(t["NUM"]) == 0:
                continue
            sign = t["SIGN"]
            val =  _multNums(t["NUM"])
            t["VAR"].sort()
            if len(t["VAR"])>0:
                key = "*".join(t["VAR"])
            else:
                key = "NUM"
            if key in termMap:
                oldsign, oldval = termMap[key]
                newval = oldsign * oldval + sign * val
                sign = 1 if newval > 0 else -1
                val = newval * sign
            termMap[key] = (sign, val)

        keys = sorted([(-_getDegrees(k), k) for k,_ in termMap.items() if k != "NUM" ])

        varTerms = [ {"VAR": [k], "NUM": [termMap[k][1]], "SIGN": termMap[k][0],} for _, k in keys if termMap[k][1] != 0]
        numTerms = [ {"VAR": [], "NUM": [termMap["NUM"][1]], "SIGN": termMap["NUM"][0],} ]
        return varTerms + numTerms

    def _multTerms(terms1, terms2):
        """
         terms1: a b c
         terms2: 1 2 3
            1 * a + 2 * a + 3 * a + 
            1 * b + 2 * b + 3 * b + 
            1 * c + 2 * c + 3 * c
        """
        allterms = []

        # print(f"{terms1=}")
        # print(f"{terms2=}")        
        for t1 in terms1:
            for t2 in terms2:
                #4 cases: var * var, var * num, num * var, num * num
                allvars = t1["VAR"] + t2["VAR"]
                allnums = [_multNums(t1["NUM"]) * _multNums(t2["NUM"])]
                sign = t1["SIGN"] * t2["SIGN"]
                if allnums[0] == 0:
                    continue
                allterms.append({"VAR": allvars, "NUM": allnums, "SIGN": sign,})

                # print("\t", t1)
                # print("\t", t2)
                # print("\t", allterms)
    
        return _addTerms(allterms)

    def _readOut(terms):
        """ assume sorted in the right order """

        retVal = []
        for t in terms:
            if len(t["VAR"]):
                retVal.append( str(t["SIGN"] * _multNums(t["NUM"])) + "*" + "*".join(t["VAR"]) )
            else:
                retVal.append( str(t["SIGN"] * _multNums(t["NUM"])) )
        return retVal

    varMap = {}
    for var, val in zip(evalvars, evalints):
        varMap[var] = val

    stack = [] #  (prevTerms, prevOp)
    terms = []
    cache = {"VAR": [], "NUM": [], "SIGN": 1,}
    prevOp = ""
    for t in s.replace("(", " ( ").replace(")", " ) ").split():
        if t == "+" or t == "-":
            # push cache to terms
            terms.append(cache)
            cache = {"VAR": [], "NUM": [], "SIGN": 1,}
            prevOp = t
        elif t == "*":
            prevOp = t
        elif t == "(":
            # use stack to save/load state
            # print("Saving ")
            # print(f"\t{terms=}")
            # print(f"\t{cache=}")
            # print(f"\t{prevOp=}")
            stack.append(
                ( _addTerms(terms), cache, prevOp )
            )
            terms = []
            cache = {"VAR": [], "NUM": [], "SIGN": 1,}
            prevOp = ""

        elif t == ")":

            if len(cache["VAR"]) + len(cache["NUM"]) > 0:
                terms.append(cache)
            curTerms = _addTerms(terms)

            # print("CurTerms ")
            # print(f"\t{curTerms=}")            
            # pop prevState
            # prevState prevOp curState -> add back to terms
            prevTerms, prevCache, op = stack.pop()
            # print("prevTerms ")
            # print(f"\t{prevTerms=}")
            # print(f"\t{prevCache=}")
            # print(f"\t{op=}")            
            if len(prevCache["VAR"]) + len(prevCache["NUM"]) > 0 and op == "*":
                # print("PrevTerms + prevCache * curTerms")
                terms = _addTerms(  prevTerms,
                                    _multTerms([prevCache], curTerms) )
                cache = {"VAR": [], "NUM": [], "SIGN": 1,}
                prevOp = ""            

            elif op == "*":
                # print("PrevTerms * curTerms")
                terms = _multTerms(prevTerms, curTerms)
                cache = {"VAR": [], "NUM": [], "SIGN": 1,}
                prevOp = ""            
            else:
                # print("PrevTerms + prevCache + curTerms")
                if len(prevCache["VAR"]) + len(prevCache["NUM"]) > 0:
                    prevTerms.append(prevCache)

                if op == "-":
                    for t in curTerms:
                        t["SIGN"] *= -1

                terms = _addTerms(prevTerms + curTerms)
                cache = {"VAR": [], "NUM": [], "SIGN": 1,}
                prevOp = ""            
        
        elif _isNum(t):
            _num = int(t)
            cache["NUM"].append(_num)
            if prevOp == "-":
                cache["SIGN"] *= -1
            prevOp = ""

        elif t in varMap:
            _num = varMap[t]
            if prevOp == "-":
                _num *= -1
            cache["NUM"].append(_num)
            prevOp = ""

        else:
            cache["VAR"].append(t)
            if prevOp == "-":
                cache["SIGN"] *= -1
            prevOp = ""

    # push cache to terms
    if len(cache["VAR"]) + len(cache["NUM"]) > 0:
        terms.append(cache)
    # for t in terms:
    #     print(t)
    # print(_addTerms(terms))


    return _readOut(_addTerms(terms))


tests = [
(                 "e + 8 - a + 5",                ["e"],     [1],        ["-1*a",  "14"]),
(                 "e * 8 - a + 5",                ["e"],     [1],        ["-1*a",  "13"]),
(                 "e + 8 - a - 5",                ["e"],     [1],        ["-1*a",  "4"]),
(                 "e * 8 - a - 5",                ["e"],     [1],        ["-1*a",  "3"]),
("e - 8 + temperature - pressure", ["e", "temperature"], [1, 12], ["-1*pressure",   "5"]),
(             "(e + 8) * (e - 8)",                   [],      [],       ["1*e*e", "-64"])
]
for t in tests:
    retVal = calc(t[0], t[1], t[2])
    print(t, retVal)
    assert(retVal == t[3])

('e + 8 - a + 5', ['e'], [1], ['-1*a', '14']) ['-1*a', '14']
('e * 8 - a + 5', ['e'], [1], ['-1*a', '13']) ['-1*a', '13']
('e + 8 - a - 5', ['e'], [1], ['-1*a', '4']) ['-1*a', '4']
('e * 8 - a - 5', ['e'], [1], ['-1*a', '3']) ['-1*a', '3']
('e - 8 + temperature - pressure', ['e', 'temperature'], [1, 12], ['-1*pressure', '5']) ['-1*pressure', '5']
('(e + 8) * (e - 8)', [], [], ['1*e*e', '-64']) ['1*e*e', '-64']
